In [3]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [6]:
client.search_experiments()

[<Experiment: artifact_location='file:///c:/Users/BazB1/Desktop/Work/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1716907076632, experiment_id='1', last_update_time=1716907076632, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/BazB1/Desktop/Work/mlops-zoomcamp/02-experiment-tracking/mlruns/0', creation_time=1716907076619, experiment_id='0', last_update_time=1716907076619, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
client.create_experiment("experiment_1")

'2'

### Search through the runs using queries

In [15]:
from mlflow.entities import ViewType

runs = client.search_runs(
        experiment_ids='1', # nyc-taxi-experiment
        filter_string="metrics.rmse < 6.33",
        run_view_type=ViewType.ACTIVE_ONLY, # Only keep active runs
        max_results=5,
        order_by=["metrics.rmse ASC"]
    )

### Print the top 5 results ranked by RMSE

In [16]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 44cbcc06bc824d469001a42fc10aac96, rmse: 6.3278
run id: 43fd694bf27b486388ce447743d88705, rmse: 6.3278
run id: a8a7b168d47e497fbe9e416a178488fa, rmse: 6.3278


### Promote Models to Model Registry

In [17]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [18]:
run_id = "a8a7b168d47e497fbe9e416a178488fa"
model_uri = f"runs:/{run_id}/model"

mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regression")

Registered model 'nyc-taxi-regression' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regression'.


<ModelVersion: aliases=[], creation_timestamp=1717102770061, current_stage='None', description=None, last_updated_timestamp=1717102770061, name='nyc-taxi-regression', run_id='a8a7b168d47e497fbe9e416a178488fa', run_link=None, source='file:///c:/Users/BazB1/Desktop/Work/mlops-zoomcamp/02-experiment-tracking/mlruns/1/a8a7b168d47e497fbe9e416a178488fa/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

### Transition from one model stage to another (deprecated)

In [20]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1717102395373, description='', last_updated_timestamp=1717102770061, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1717102770061, current_stage='None', description=None, last_updated_timestamp=1717102770061, name='nyc-taxi-regression', run_id='a8a7b168d47e497fbe9e416a178488fa', run_link=None, source='file:///c:/Users/BazB1/Desktop/Work/mlops-zoomcamp/02-experiment-tracking/mlruns/1/a8a7b168d47e497fbe9e416a178488fa/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>,
  <ModelVersion: aliases=[], creation_timestamp=1717102612040, current_stage='Staging', description='', last_updated_timestamp=1717102634278, name='nyc-taxi-regression', run_id='43fd694bf27b486388ce447743d88705', run_link='', source='models:/nyc-taxi-regression/2', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regression', tags={}>]

In [22]:
model_name = "nyc-taxi-regression"

latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 4, stage: None
version: 3, stage: Staging


C:\Users\BazB1\AppData\Local\Temp\ipykernel_32136\238268840.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [24]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

C:\Users\BazB1\AppData\Local\Temp\ipykernel_32136\788098555.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1717102770061, current_stage='Staging', description=None, last_updated_timestamp=1717103444205, name='nyc-taxi-regression', run_id='a8a7b168d47e497fbe9e416a178488fa', run_link=None, source='file:///c:/Users/BazB1/Desktop/Work/mlops-zoomcamp/02-experiment-tracking/mlruns/1/a8a7b168d47e497fbe9e416a178488fa/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

### Annotate models

In [27]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to stage {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1717102770061, current_stage='Staging', description='The model version 4 was transitioned to stage Staging on 2024-05-30', last_updated_timestamp=1717103518764, name='nyc-taxi-regression', run_id='a8a7b168d47e497fbe9e416a178488fa', run_link=None, source='file:///c:/Users/BazB1/Desktop/Work/mlops-zoomcamp/02-experiment-tracking/mlruns/1/a8a7b168d47e497fbe9e416a178488fa/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

### Comparing versions and selecting the new "Production" model

- In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. 
- The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

In [35]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [37]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

### Download DV preprocessor

In [38]:
run_id = "44cbcc06bc824d469001a42fc10aac96" # "a8a7b168d47e497fbe9e416a178488fa" doesn't have preprocessor artifact

client.download_artifacts(run_id=run_id, path="preprocessor", dst_path=".")

'c:\\Users\\BazB1\\Desktop\\Work\\mlops-zoomcamp\\02-experiment-tracking\\preprocessor'

In [39]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [40]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2021-03-01 00:05:42,2021-03-01 00:14:03,N,1.0,83,129,1.0,1.56,7.50,...,0.5,0.00,0.0,None,0.3,8.80,1.0,1.0,0.0,8.350000
1,2,2021-03-01 00:21:03,2021-03-01 00:26:17,N,1.0,243,235,1.0,0.96,6.00,...,0.5,0.00,0.0,None,0.3,7.30,2.0,1.0,0.0,5.233333
2,2,2021-03-01 00:02:06,2021-03-01 00:22:26,N,1.0,75,242,1.0,9.93,28.00,...,0.5,2.00,0.0,None,0.3,31.30,1.0,1.0,0.0,20.333333
3,2,2021-03-01 00:24:03,2021-03-01 00:31:43,N,1.0,242,208,1.0,2.57,9.50,...,0.5,0.00,0.0,None,0.3,10.80,2.0,1.0,0.0,7.666667
4,1,2021-03-01 00:11:10,2021-03-01 00:14:46,N,1.0,41,151,1.0,0.80,5.00,...,0.5,1.85,0.0,None,0.3,8.15,1.0,1.0,0.0,3.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83822,2,2021-03-31 22:07:00,2021-03-31 22:13:00,None,NaN,41,75,NaN,1.48,8.46,...,0.0,1.44,0.0,None,0.3,10.20,NaN,NaN,NaN,6.000000
83823,2,2021-03-31 22:56:00,2021-03-31 23:13:00,None,NaN,95,95,NaN,0.09,54.25,...,0.0,0.00,0.0,None,0.3,57.30,NaN,NaN,NaN,17.000000
83824,2,2021-03-31 22:36:00,2021-03-31 22:45:00,None,NaN,95,95,NaN,0.66,8.11,...,0.0,0.00,0.0,None,0.3,8.41,NaN,NaN,NaN,9.000000
83825,2,2021-03-31 23:35:00,2021-04-01 00:00:00,None,NaN,37,14,NaN,9.58,36.83,...,0.0,0.00,0.0,None,0.3,39.88,NaN,NaN,NaN,25.000000


In [41]:
X_test = preprocess(df, dv)

In [42]:
target = "duration"
y_test = df[target].values

### Test the models

In [44]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

c:\Users\BazB1\Desktop\Work\mlops-zoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:28:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: total: 21.6 s
Wall time: 4.98 s


c:\Users\BazB1\Desktop\Work\mlops-zoomcamp\.venv\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.272238244783205}

In [46]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

c:\Users\BazB1\Desktop\Work\mlops-zoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:32:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: total: 19.2 s
Wall time: 1.83 s


c:\Users\BazB1\Desktop\Work\mlops-zoomcamp\.venv\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.272238244783205}

### Transition to Production if model performance is better

In [47]:
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage="Production",
    archive_existing_versions=True
)

C:\Users\BazB1\AppData\Local\Temp\ipykernel_32136\4095759336.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1717102770061, current_stage='Production', description='The model version 4 was transitioned to stage Staging on 2024-05-30', last_updated_timestamp=1717112925379, name='nyc-taxi-regression', run_id='a8a7b168d47e497fbe9e416a178488fa', run_link=None, source='file:///c:/Users/BazB1/Desktop/Work/mlops-zoomcamp/02-experiment-tracking/mlruns/1/a8a7b168d47e497fbe9e416a178488fa/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>